In [5]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image
import requests
import os
import torch

# 设置代理，端口7890
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

image_path = "/workspace/测试文件夹/bedroom.jpg"
image = Image.open(image_path)
# 按照短边缩放到512，长边等比例缩放, 同时长边可以被8整除
width, height = image.size
if width < height:
    new_width = 512
    new_height = int(height * (512 / width)) // 8 * 8
    image = image.resize((new_width, new_height))
else:
    new_height = 512
    new_width = int(width * (512 / height)) // 8 * 8
    image = image.resize((new_width, new_height))

print(image.size)
# image


(816, 512)


In [6]:
# Loading a single model for all three tasks
# 加载本地模型
model_path = r"/home/public/ai_chat_data/models/oneformer_ade20k_swin_large"
processor = OneFormerProcessor.from_pretrained(model_path,local_files_only=True)
model = OneFormerForUniversalSegmentation.from_pretrained(model_path, local_files_only=True)

/opt/conda/envs/py311/lib/python3.11/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `OneFormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)
/opt/conda/envs/py311/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

# Semantic Segmentation
with torch.no_grad():
    semantic_inputs = processor(images=image, task_inputs=["semantic"], return_tensors="pt")
    semantic_outputs = model(**semantic_inputs)
    # pass through image_processor for postprocessing
    predicted_semantic_map = processor.post_process_semantic_segmentation(semantic_outputs, target_sizes=[image.size[::-1]])[0]
    predicted_semantic_map


tensor([[5, 5, 5,  ..., 0, 0, 0],
        [5, 5, 5,  ..., 0, 0, 0],
        [5, 5, 5,  ..., 0, 0, 0],
        ...,
        [3, 3, 3,  ..., 3, 3, 3],
        [3, 3, 3,  ..., 3, 3, 3],
        [3, 3, 3,  ..., 3, 3, 3]])

In [ ]:

# Instance Segmentation
instance_inputs = processor(images=image, task_inputs=["instance"], return_tensors="pt")
instance_outputs = model(**instance_inputs)
# pass through image_processor for postprocessing
predicted_instance_map = processor.post_process_instance_segmentation(outputs, target_sizes=[image.size[::-1]])[0]["segmentation"]

# Panoptic Segmentation
panoptic_inputs = processor(images=image, task_inputs=["panoptic"], return_tensors="pt")
panoptic_outputs = model(**panoptic_inputs)
# pass through image_processor for postprocessing
predicted_semantic_map = processor.post_process_panoptic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]["segmentation"]

In [25]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image
import requests
import torch

# load OneFormer fine-tuned on ADE20k for universal segmentation
processor = OneFormerProcessor.from_pretrained(model_path, local_files_only=True)
model = OneFormerForUniversalSegmentation.from_pretrained(model_path, local_files_only=True)

url = (
    "https://huggingface.co/datasets/hf-internal-testing/fixtures_ade20k/resolve/main/ADE_val_00000001.jpg"
)
image = Image.open(requests.get(url, stream=True).raw)
print(image.size)

# Semantic Segmentation
inputs = processor(image, ["semantic"], return_tensors="pt")
print(inputs.keys())
print(inputs["pixel_values"].shape)

with torch.no_grad():
    outputs = model(**inputs)  #

# model predicts class_queries_logits of shape `(batch_size, num_queries)`
# and masks_queries_logits of shape `(batch_size, num_queries, height, width)`
class_queries_logits = outputs.class_queries_logits
masks_queries_logits = outputs.masks_queries_logits

# you can pass them to processor for semantic postprocessing
predicted_semantic_map = processor.post_process_semantic_segmentation(
    outputs, target_sizes=[(image.height, image.width)]
)[0]


/opt/conda/envs/py311/lib/python3.11/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `OneFormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)
/opt/conda/envs/py311/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(683, 512)
dict_keys(['pixel_values', 'pixel_mask', 'task_inputs'])
torch.Size([1, 3, 640, 853])
odict_keys(['class_queries_logits', 'masks_queries_logits', 'auxiliary_predictions', 'encoder_hidden_states', 'pixel_decoder_hidden_states', 'transformer_decoder_hidden_states', 'transformer_decoder_object_queries', 'transformer_decoder_contrastive_queries', 'transformer_decoder_mask_predictions', 'transformer_decoder_class_predictions', 'transformer_decoder_auxiliary_predictions', 'task_token', 'attentions'])


In [18]:
predicted_semantic_map

tensor([[2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        ...,
        [9, 9, 9,  ..., 6, 6, 6],
        [9, 9, 9,  ..., 6, 6, 6],
        [9, 9, 9,  ..., 6, 6, 6]])